In [12]:
import cv2, sys
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageEnhance, ImageChops
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers import LeakyReLU

In [13]:
import os
import datetime
import time, random
from tqdm import tqdm
import glob

In [141]:
# 테스트 데이터 꺼내오기 
# 독립문 훈련, 테스트셋 나누기 
door_folder = "C:/source/image/indep_door"
park_folder = "C:/source/image/tapgol_park"
door_file_list = [x for x in os.listdir(door_folder)]
park_file_list = [x for x in os.listdir(park_folder)]

In [142]:
door_test_list = door_file_list[:200]
len(door_test_list)

200

In [143]:
door_train_list = door_file_list[200:]
len(door_train_list)

136

In [144]:
# 독립문 테스트셋
file_name_freq  = 0
file_name_freq += 1 
for i in tqdm(range(len(door_test_list))):
    img = load_img("C:/source/image/indep_door/" + door_test_list[i])
    img = img.resize((300, 300))
    img.save("C:/source/image/test/indep_door/" + door_test_list[i])

100%|███████████████████████████████████████████████████████████████████████████| 200/200 [00:14<00:00, 14.08it/s]


In [145]:
# 독립문 훈련셋
for i in tqdm(range(len(door_train_list))):
    img = load_img("C:/source/image/indep_door/" + door_train_list[i])
    img = img.resize((300, 300))
    img.save("C:/source/image/train/indep_door/" + door_train_list[i])
    

100%|███████████████████████████████████████████████████████████████████████████| 136/136 [00:10<00:00, 12.69it/s]


In [146]:
park_test_list = park_file_list[:200]
park_train_list = park_file_list[200:]

In [147]:
# 탑골공원 테스트셋
for i in tqdm(range(len(park_test_list))):
    img = load_img("C:/source/image/tapgol_park/" + park_test_list[i])
    img = img.resize((300, 300))
    img.save("C:/source/image/test/tapgol_park/" + park_test_list[i])

100%|███████████████████████████████████████████████████████████████████████████| 200/200 [00:19<00:00, 10.16it/s]


In [148]:
# 탑골공원 훈련셋
for i in tqdm(range(len(park_train_list))):
    img = load_img("C:/source/image/tapgol_park/" + park_train_list[i])
    img = img.resize((300, 300))
    img.save("C:/source/image/train/tapgol_park/" + park_train_list[i])

100%|███████████████████████████████████████████████████████████████████████████| 605/605 [00:57<00:00, 10.61it/s]


In [185]:
# 학습이미지 부풀리기
datagen = ImageDataGenerator(\
                            rotation_range=10, # 정수범위만큼 무작위 회전
                            width_shift_range=0.1,  # 픽셀단위로 좌우이동
                            height_shift_range=0.1, # 픽셀단위로 상하이동
                            shear_range=0.5, # 이미지 찌그러트리기
                            zoom_range=0.2, # 이미지 확대 축소 
                            horizontal_flip=True, # 좌우반전
                            fill_mode="nearest") # 이미지에 여백 생겼을 때

In [186]:
# 학습이미지만 부풀리기
# 독립문 
file_name_freq = 0
for i in tqdm(range(len(door_train_list))):
    img = load_img("C:/source/image/train/indep_door/" + door_train_list[i])  # PIL 이미지
    x = img_to_array(img) # (3, 300, 300) 크기의 Numpy 배열
    x = x.reshape((1,) + x.shape) # (1, 3, 300, 300) 크기의 Numpy 배열
    file_name_freq += 1
    
    # 변환된 이미지를 배치 단위로 생성해서 지정된 폴더에 저장
    i = 0
    for batch in datagen.flow(x, batch_size=1,\
                             save_to_dir="C:/source/image/train/indep_door/",\
                             save_prefix=door_train_list[i] + str(file_name_freq),\
                             save_format="jpg"):
        i += 1
        if i > 6:
            break # 이미지 한 개 당 6개씩 부풀리기

  0%|                                                                                     | 0/136 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:/source/image/train/indep_door/HF010435_0103_0089.JPG'

In [192]:
# # 학습이미지만 부풀리기
# # 탑골공원 
# file_name_freq = 0
# for i in tqdm(range(len(park_train_list))):
#     img = load_img("C:/source/image/train/tapgol_park/" + park_train_list[i])  # PIL 이미지
#     x = img_to_array(img) # (3, 300, 300) 크기의 Numpy 배열
#     x = x.reshape((1,) + x.shape) # (1, 3, 300, 300) 크기의 Numpy 배열
#     file_name_freq += 1
    
#     # 변환된 이미지를 배치 단위로 생성해서 지정된 폴더에 저장
#     i = 0
#     for batch in datagen.flow(x, batch_size=1,\
#                              save_to_dir="C:/source/image/train/tapgol_park/",\
#                              save_prefix=park_train_list[i] + str(file_name_freq),\
#                              save_format="jpg"):
#         i += 1
#         if i > 2:
#             break # 이미지 한 개 당 6개씩 부풀리기

In [2]:
# 데이터 훈련시키기 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics


In [3]:
b_size=5

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('D:/image/train/',target_size=(200,200),
                                                    batch_size=b_size, class_mode='categorical')

Found 1440 images belonging to 2 classes.


In [8]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory('D:/image/validation/',target_size=(200,200),
                                                    batch_size=b_size, class_mode='categorical')

Found 480 images belonging to 2 classes.


In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('D:/image/test/',target_size=(200,200),
                                                    batch_size=b_size, class_mode='categorical')

Found 480 images belonging to 2 classes.


In [14]:
model = Sequential()
model.add(Conv2D(20, (3, 3), input_shape=(200,200,3)))
model.add(Activation(LeakyReLU(alpha=0.01)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(20, (3, 3)))
model.add(Activation(LeakyReLU(alpha=0.01)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(40, (3, 3)))
model.add(Activation(LeakyReLU(alpha=0.01)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(40))
model.add(Activation(LeakyReLU(alpha=0.01)))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 198, 198, 20)      560       
_________________________________________________________________
activation (Activation)      (None, 198, 198, 20)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 20)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 97, 97, 20)        3620      
_________________________________________________________________
activation_1 (Activation)    (None, 97, 97, 20)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 20)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 46, 46, 40)       

In [16]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=0.0002),\
              metrics=['accuracy'])

In [17]:
steps_train = len(train_generator)
steps_test = len(test_generator)

In [18]:
history = model.fit(
       train_generator,
       steps_per_epoch=steps_train,
       epochs=20,
       validation_data=test_generator,
       validation_steps=steps_test )

Train for 288 steps, validate for 96 steps
Epoch 1/20
288/288 [==============================] - 6s 22ms/step - loss: 0.1550 - accuracy: 0.9361 - val_loss: 0.0268 - val_accuracy: 0.9958
Epoch 2/20
288/288 [==============================] - 4s 15ms/step - loss: 0.0365 - accuracy: 0.9882 - val_loss: 0.0190 - val_accuracy: 0.9958
Epoch 3/20
288/288 [==============================] - 4s 15ms/step - loss: 0.0168 - accuracy: 0.9958 - val_loss: 0.0146 - val_accuracy: 0.9958
Epoch 4/20
288/288 [==============================] - 4s 15ms/step - loss: 0.0183 - accuracy: 0.9958 - val_loss: 0.0226 - val_accuracy: 0.9958
Epoch 5/20
288/288 [==============================] - 4s 15ms/step - loss: 0.0120 - accuracy: 0.9972 - val_loss: 0.0096 - val_accuracy: 0.9979
Epoch 6/20
288/288 [==============================] - 4s 15ms/step - loss: 0.0044 - accuracy: 0.9986 - val_loss: 0.0069 - val_accuracy: 0.9958
Epoch 7/20
288/288 [==============================] - 4s 15ms/step - loss: 0.0047 - accuracy: 0.999

In [19]:
new_model = Sequential()
new_model.add(Conv2D(20, (3, 3), input_shape=(300,300,3)))
model.add(Activation(LeakyReLU(alpha=0.01)))
new_model.add(MaxPooling2D(pool_size=(2, 2)))

new_model.add(Conv2D(20, (3, 3)))
model.add(Activation(LeakyReLU(alpha=0.01)))
new_model.add(MaxPooling2D(pool_size=(2, 2)))

new_model.add(Conv2D(40, (3, 3)))
model.add(Activation(LeakyReLU(alpha=0.01)))
new_model.add(MaxPooling2D(pool_size=(2, 2)))

new_model.add(Flatten())
new_model.add(Dense(40))
model.add(Activation(LeakyReLU(alpha=0.01)))
new_model.add(Dropout(0.5))
new_model.add(Dense(2))
new_model.add(Activation('softmax'))

In [20]:
new_model.load_weights('weight/jinsu_weight.ckpt')


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000219E41AE588> and <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x00000219E4FBFFD0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000219E4220898> and <tensorflow.python.keras.layers.core.Flatten object at 0x00000219E4FAB780>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000218E38147B8> and <tensorflow.python.keras.layers.core.Activation object at 0x00000218E3842940>).


ValueError: Shapes (49000, 40) and (21160, 40) are incompatible

In [47]:
new_model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=0.0002),\
              metrics=['accuracy'])

In [48]:
new_model.evaluate(test_generator)

96/96 [==============================] - 2s 17ms/step - loss: 37.5844 - accuracy: 0.6250


[37.58441987876631, 0.625]